In [2]:
## all imports
from IPython.display import HTML
import numpy as np
from urllib import request # urllib2 isn't supported by P3 - https://stackoverflow.com/questions/58794540/no-module-named-urllib2-how-do-i-use-it-in-python-so-i-can-make-a-request
import bs4 #this is beautiful soup
import time
import operator
import socket
import pickle # was CPickle, but again use pickly for P3
import re # regular expressions
import csv
from datetime import date as d
from time import sleep

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

from secrets import *

In [3]:
url_string = 'https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=STATION%5E1400&minBedrooms=4&maxPrice=6000&minPrice=3000&radius=1.0&propertyTypes=&includeLetAgreed=false&mustHave=&dontShow=&furnishTypes=&keywords='

print("You've scraped the following pages: \n")

i = 0
total_count = 0
while(True):
    try:
        if i == 0:
            url = url_string
        else:
            url = url_string+'&index='+format(i)
        source = request.urlopen(url).read().decode()
        soup = bs4.BeautifulSoup(source, 'lxml')

        #rest of the scraping code
        
        # get the headline (i.e. the number of rooms and type of building):
        title = [title.text.strip() for title in soup.findAll('h2', {'class': 'propertyCard-title'})] # list comprehension woop! So I believe that the text attribute is specific to BS?
        # get the address:
        address = [address['content'] for address in soup.findAll('meta', {'itemprop': 'streetAddress'})] #need only the content element
        #get the number of miles from the station in question:
        miles = [miles.text.strip() for miles in soup.findAll('div', {'class': 'propertyCard-distance'})]
        miles = [float(re.findall("\d+\.\d+", i)[0]) for i in miles]
        #get the property description:
        desc = [desc.text.strip() for desc in soup.findAll('span', {'itemprop':'description'})]
        #get the date added to the site:
        dateadded = [dateadded.text.strip() for dateadded in soup.findAll('span', {'class': 'propertyCard-branchSummary-addedOrReduced'})]
        #get the estate agent's name:
        agent = [agent.text.strip() for agent in soup.findAll('span', {'class': 'propertyCard-branchSummary-branchName'})]
        #get the agent's number:
        number = [number.text.strip() for number in soup.findAll('a', {'class': 'propertyCard-contactsPhoneNumber'})]
        #get the property's price:
        price = [price.text.strip() for price in soup.findAll('span', {'class': 'propertyCard-priceValue'})]
        # link to the property's page:
        link = ['https://www.rightmove.co.uk' + link['href'] for link in soup.findAll('a', {'class': 'propertyCard-headerLink'})]

        count = sum(1 for i in price if not i is '')
        if count == 0:
            break
        else:
            pass
        
        total_count += count
        
        #print page url
        print(url+'\n')
        
        if i == 0:
            results = []
        else:
            pass

        for index in range(0, count):
            results.append({
                'date_added to file' : d.today().strftime("%d/%m/%Y"), #https://www.programiz.com/python-programming/online-compiler/?ref=957b4f50
                'title' : title[index],
                'address' : address[index],
                'miles from station' : miles[index],
                'description' : desc[index],
                'price' : price[index],
                'date added/price changed' : dateadded[index],
                'agent' : agent[index],
                'agent phone number' : number[index],
                'link to property' : link[index]
            })
        
        
        #don't overflow website
        sleep(2)

        
        #increase page number
        i += 24
    except:
        break
        
print("You have the data on: "+str(total_count)+" properties")

You've scraped the following pages: 

https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=STATION%5E1400&minBedrooms=4&maxPrice=6000&minPrice=3000&radius=1.0&propertyTypes=&includeLetAgreed=false&mustHave=&dontShow=&furnishTypes=&keywords=

You have the data on: 23 properties


In [4]:
df = pd.DataFrame.from_dict(results)
df.to_csv('properties.csv',encoding='utf-8', mode='w')
df.to_csv('properties_cumulative.csv',encoding='utf-8', mode='a')